In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import escalateclient
import importlib

First, login to the escalate server where your data is hosted. In this case we are logging in to a local instance of the webserver running on port 8000

In [2]:
importlib.reload(escalateclient)
server_url = 'http://localhost:8000'
username = 'vshekar'
password = 'copperhead123'
client = escalateclient.ESCALATEClient(server_url, username, password)

In [6]:
# Get the Nielson lab
nielson_lab_response = client.get(endpoint='actor', data={'description': 'Nielson Lab', 'person__isnull': True, 'systemtool__isnull': True})

GET: OK
Found one resource, returning dict


In [ ]:
active_status = client.get_or_create(endpoint='status', data={'description': 'active'})

# Inventory details
nl_inventory = {
    "description": "Nielson Lab Inventory",
    "status": active_status[0]['url'], # Indicates active status
    "actor": nielson_lab_response[0]['url'],
    "owner": nielson_lab_response[0]['url'], # Indicates James Neilson as owner
    "operator": nielson_lab_response[0]['url'], # Indicates James Neilson as operator
    "lab": nielson_lab_response[0]['url'] # Associates inventory with Neilson Lab
}

# Get lab inventory
nl_inventory_response = client.get_or_create(endpoint='inventory', data=nl_inventory)

In [ ]:
# Load materials into inventory

import pandas as pd 
df = pd.read_csv('MetalChalcogenideRxns.csv')
#Add material to db
data = {
    'description': '',
    'consumable': True,
    'material_class': 'model',
}

for chemical in df['Product expected']:
    data['description'] = chemical
    client.post('material', data=data)

for i, row in df.iterrows():
    material_name = row['Product expected']
    # Check that material is in db (add it if it isn't)
    material_data = {'description': material_name, 
                        'consumable': True,
                        'material_class':'model'}
    material_response = client.get_or_create('material', data=material_data)
    # Add material to inventory
    im_data = {
        "description": material_name,
        "phase": "solid",
        "inventory": nl_inventory_response[0]['url'],
        "material": material_response[0]['url']
    }
    im_response = client.get_or_create('inventory-material', data=im_data)